In [1]:
#Required packages
import numpy as np
import pandas as pd
from riotwatcher import LolWatcher, ApiError
import requests
from bs4 import BeautifulSoup

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

import xgboost as xgb
import time

In [2]:
gameIDs = [3450551207,
 3450447940,
 3450473349,
 3449910866,
 3449809573,
 3449805002,
 3449395985,
 3449420612,
 3449368014,
 3448536602,
 3409538636,
 3409524617,
 3409469635,
 3409444191,
 3406092238,
 3406038402,
 3406044245,
 3405683989,
 3405672616,
 3405659791,
 3449539255,
 3449526251,
 3449560875,
 3449497184,
 3449464285,
 3449480180,
 3449415869,
 3449391063,
 3449327412,
 3449314380,
 3450473349,
 3450384967,
 3449941896,
 3449917039,
 3449911789,
 3449857306,
 3449804628,
 3449616597,
 3449619982,
 3449403502,
 4645085775,
 4645101169,
 4644995725,
 4644953842,
 4644300302,
 4644256957,
 4644042414,
 4643988234,
 4643935599,
 3450509221,
 3450534377,
 3450124069,
 3449844179,
 3449750568,
 3449712500,
 3449616932,
 3449415798,
 3449033379,
 3449031891,
 3450392647,
 3450369819,
 3450298553,
 3449327413,
 3449362393,
 3449308674,
 3449304193,
 3449271685,
 3447796176,
 3447244796,
 3433285002,
 3433331078,
 3432025961,
 3431090080,
 3430975462,
 3429731288,
 3429676809,
 3429692607,
 3429627604,
 3429613210,
 3450375943,
 3449866243,
 3449830979,
 3449776002,
 3449495026,
 3449439091,
 3449431370,
 3449377274,
 3449047030,
 3449025867,
 3450005974,
 3449964440,
 3449962378,
 3449979813,
 3449878695,
 3449925458,
 3449893318,
 3449880432,
 3449816848,
 3449812809,
 3450695188,
 3450568282,
 3450544556,
 3450479281,
 3450464980,
 3450419606,
 3450406026,
 3450362904,
 3450308464,
 3450304095,
 3450745565,
 3450528870,
 3450523067,
 3450448097,
 3450452620,
 3450427845,
 3450374282,
 3450309216,
 3449884861,
 3449880179,
 3450731007,
 3450516635,
 3450560309,
 3450444135,
 3450419044,
 3449813490,
 3449777276,
 3449728879,
 3449692727,
 3449606090,
 3450741213,
 3450563435,
 3450488720,
 3450461817,
 3450394864,
 3450327184,
 3450351829,
 3450248109,
 3450234512,
 3450279835,
 3450694709,
 3449846600,
 3449759709,
 3449785591,
 3449783099,
 3449676763,
 3449617728,
 3449588224,
 3448755401,
 3448697939,
 3450558448,
 3450505068,
 3450445944,
 3450407165,
 3449726853,
 3449660282,
 3449630752,
 3449554104,
 3449476231,
 3449472689,
 3450545098,
 3450560412,
 3450446586,
 3450473198,
 3450430925,
 3450385550,
 3450373168,
 3450329699,
 3450326630,
 3450294957,
 3450425241,
 3449329223,
 3448492049,
 3447286387,
 3447331395,
 3447226619,
 3447270771,
 3447176125,
 3447163078,
 3444290928,
 3450484230,
 3450428956,
 3450366184,
 3450420696,
 3450212645,
 3450152504,
 3450099561,
 3449943780,
 3449939374,
 3449907297,
 3450125353,
 3450027171,
 3450016521,
 3448744211,
 3447640325,
 3447500845,
 3447486927,
 3446684221,
 3445300705,
 3445209111,
 3441949792,
 3441835026,
 3441768640,
 3441774007,
 3441708979,
 3440582524,
 3440512004,
 3440451577,
 3440392919,
 3440317855,
 3450510962,
 3450423204,
 3450358804,
 3449793135,
 3449664766,
 3449669795,
 3449518832,
 3449485268,
 3449364610,
 3449291761,
 3449884457,
 3448980336,
 3448938359,
 3448936508,
 3448943315,
 3448930796,
 3446547041,
 3446525925,
 3446397456,
 3446374894,
 3450688373,
 3450730701,
 3450547586,
 3450534834,
 3450312656,
 3450269435,
 3450264751,
 3450232604,
 3450198826,
 3450157099,
 3450764142,
 3450539066,
 3450505676,
 3450529812,
 3450381020,
 3450324993,
 3449571245,
 3449482693,
 3449417483,
 3449392511,
 3450296009,
 3450331157,
 3450268806,
 3450256812,
 3450234064,
 3449839851,
 3449736843,
 3449733639,
 3449739744,
 3449714184,
 3450476305,
 3450452826,
 3450407999,
 3450413335,
 3450026065,
 3450014773,
 3448971065,
 3448958123,
 3448916971,
 3448953924,
 3450520211,
 3450435994,
 3450429716,
 3450409866,
 3449952414,
 3449873056,
 3449825293,
 3449800973,
 3449752536,
 3449668619,
 3450727119,
 3450711215,
 3450679737,
 3450554156,
 3450531375,
 3450458254,
 3449843707,
 3449802689,
 3449757189,
 3449482344,
 3450720349,
 3450296282,
 3450279449,
 3449337084,
 3448482102,
 3448386131,
 3448369912,
 3447006078,
 3447012138,
 3446531781,
 3449830365,
 3448810451,
 3448698614,
 3448715207,
 3447498244,
 3447397040,
 3446483156,
 3446157945,
 3446134170,
 3445482118,
 3450763131,
 3450509379,
 3450043298,
 3450022708,
 3450021085,
 3449997452,
 3450005423,
 3449991818,
 3449929326,
 3449834867,
 3444774465,
 3443699955,
 3443565970,
 3441677122,
 3441643503,
 3441650198,
 3441547629,
 3441604819,
 3441459378,
 3441467015,
 3449963410,
 3448942313,
 3448901909,
 3448900221,
 3448838483,
 3448846324,
 3448852170,
 3448809451,
 3448815395,
 3447572474,
 3450723655,
 3450681360,
 3450144174,
 3449731476,
 3449676701,
 3449548839,
 3449573951,
 3449091822,
 3448657733,
 3448562611,
 3450088004,
 3449034675,
 3448907193,
 3448952594,
 3447665392,
 3447693784,
 3446621356,
 3446580862,
 3446528844,
 3445247124,
 3450722727,
 3450507414,
 3450511483,
 3450490867,
 3450346605,
 3450247756,
 3450286032,
 3450231006,
 3450176517,
 3450154237,
 3409993861,
 3409624015,
 3350921432,
 3350919797,
 3350904782,
 3350873936,
 3350881316,
 3350785397,
 3320366997,
 3319663126,
 3449971649,
 3449917823,
 3449894716,
 3449823393,
 3449767547,
 3446628851,
 3446636179,
 3445940102,
 3445725435,
 3445578925,
 3450052820,
 3450030475,
 3449962317,
 3449895352,
 3449844039,
 3449730467,
 3449404599,
 3449359009,
 3448907652,
 3448895958,
 3448907101,
 3448931831,
 3447722567,
 3446405640,
 3446409911,
 3443983563,
 3443942075,
 3443878893,
 3443855510,
 3443583056,
 3450686420,
 3450559563,
 3450488301,
 3450413877,
 3450083144,
 3450139875,
 3450068609,
 3450046779,
 3450064600,
 3450023939,
 4644943422,
 4644858002,
 4644795872,
 4644548558,
 4644219518,
 4644235258,
 4643831018,
 4643727633,
 4643282822,
 4643150114,
 3448628478,
 3448645371,
 3448568503,
 3448517971,
 3448471526,
 3448365766,
 3446470795,
 3446378802,
 3446413792,
 3446369570,
 3449984062,
 3450000667,
 3449878169,
 3449807107,
 3449800076,
 3449790637,
 3448937027,
 3448954386,
 3448892986,
 3448867964,
 3450121329,
 3450079873,
 3450036654,
 3450031482,
 3450030011,
 3449968134,
 3449965879,
 3449951492,
 3449914440,
 3449852907,
 3430699236,
 3430734039,
 3430688891,
 3430615164,
 3430552376,
 3430536793,
 3426908825,
 3426689258,
 3426644672,
 3426566361,
 3450695758,
 3450509476,
 3450513330,
 3450486853,
 3450443363,
 3450394894,
 3450410931,
 3450328332,
 3450249114,
 3450254671,
 3450175464,
 3450202804,
 3450181647,
 3450199755,
 3449639987,
 3449555587,
 3449468014,
 3449273402,
 3449250874,
 3449217310,
 3450747862,
 3450732314,
 3450546146,
 3450530413,
 3450486662,
 3450437045,
 3450463196,
 3450387815,
 3450039121,
 3450047069,
 3449854437,
 3449476744,
 3449452474,
 3447889511,
 3447894252,
 3446456157,
 3446482872,
 3446420916,
 3446355792,
 3446287464,
 3450186952,
 3450172841,
 3450150748,
 3449976688,
 3449944986,
 3449969916,
 3449877699,
 3449658188,
 3449610673,
 3449542338,
 3450199034,
 3449805753,
 3449801023,
 3449780376,
 3449685290,
 3449466938,
 3449481557,
 3448880702,
 3448788529,
 3448785342,
 3267528465,
 3267495429,
 3267349459,
 3267305538,
 3267349816,
 3267187143,
 3266800512,
 3266719253,
 3266756267,
 3266742689,
 3450735971,
 3450557007,
 3450416119,
 3449952467,
 3449859538,
 3449845595,
 3449758819,
 3449770857,
 3449534392,
 3449455915,
 3450093525,
 3450048438,
 3450017728,
 3450035931,
 3450015744,
 3450052961,
 3450042397,
 3450021585,
 3449998930,
 3449956668,
 3449977853,
 3449994910,
 3449952149,
 3449927921,
 3449922144,
 3449836804,
 3449821790,
 3449748541,
 3449719406,
 3449662101,
 3450682413,
 3450555543,
 3450530627,
 3450482597,
 3450416013,
 3450329701,
 3450286398,
 3450223601,
 3450178663,
 3450176435,
 3450457258,
 3450473486,
 3449805555,
 3449830649,
 3449783746,
 3449694037,
 3449648983,
 3449621540,
 3449544841,
 3449310734,
 3450519970,
 3450387346,
 3450347655,
 3450226011,
 3450197040,
 3450213229,
 3450170413,
 3450103774,
 3450068782,
 3450077081,
 3450568213,
 3450551607,
 3450448244,
 3450471877,
 3450269644,
 3449743349,
 3449592644,
 3449545995,
 3449541868,
 3449457898,
 3450716258,
 3450761399,
 3450545781,
 3450520668,
 3450486569,
 3450472642,
 3450310667,
 3450127309,
 3449103764,
 3448313043,
 3450542913,
 3450478039,
 3450424862,
 3450419765,
 3449878315,
 3449882757,
 3449856218,
 3449798312,
 3449728463,
 3449691983,
 3450041697,
 3449967195,
 3449991516,
 3449916246,
 3448895676,
 3448912079,
 3448876067,
 3447602972,
 3447562560,
 3447569737,
 3449938364,
 3449856342,
 3449394262,
 3448782303,
 3448586731,
 3448600886,
 3448495203,
 3448470209,
 3448113640,
 3447986263,
 3450152925,
 3450033473,
 3450041205,
 3449999302,
 3449956414,
 3449919334,
 3449914846,
 3449883599,
 3449870708,
 3449232860,
 3450713733,
 3450528520,
 3450391119,
 3450325224,
 3450302558,
 3450255599,
 3450242618,
 3450191094,
 3450108249,
 3450115882,
 3365718214,
 3320302100,
 3320291302,
 3320279194,
 3320288047,
 3320287211,
 3320286445,
 3320266640,
 3320284625,
 3320245361,
 3450506438,
 3450530182,
 3450435122,
 3450383997,
 3450318996,
 3450334347,
 3450329807,
 3450284621,
 3450222672,
 3450197520,
 3450563274,
 3450488252,
 3450485135,
 3450167373,
 3450113736,
 3450083447,
 3449428490,
 3449386983,
 3449421590,
 3449347843,
 3450327774,
 3450296407,
 3450332958,
 3450349832,
 3450256789,
 3450283671,
 3450178853,
 3450182278,
 3450089607,
 3450096192,
 3449811397,
 3449790911,
 3449706003,
 3449552791,
 3449484538,
 3449316152,
 3449329983,
 3449283736,
 3447991750,
 3447979797,
 3450512406,
 3450478342,
 3450212546,
 3450200165,
 3450159947,
 3449622008,
 3448819844,
 3448813445,
 3448727307,
 3448650487,
 3450560323,
 3450296754,
 3450259912,
 3449547523,
 3449177008,
 3449129355,
 3449089099,
 3447221678,
 3446918225,
 3446825427,
 3450184761,
 3450154653,
 3450163232,
 3450054889,
 3450014827,
 3450042577,
 3450060466,
 3449979660,
 3449893676,
 3449859720,
 4644057104,
 4644062147,
 4643973445,
 4643607701,
 4643604468,
 4642795429,
 4642659478,
 4642656339,
 4642611936,
 4642556661,
 3450499544,
 3450406372,
 3450374170,
 3449834297,
 3449769715,
 3449745405,
 3449726666,
 3449720667,
 3449607572,
 3449578026,
 3450697393,
 3450710761,
 3450554153,
 3450496801,
 3450491160,
 3450397764,
 3450402772,
 3450326475,
 3450322615,
 3450287451,
 3450423246,
 3450055410,
 3449125460,
 3449100541,
 3449012735,
 3448962009,
 3447850960,
 3447832447,
 3447766403,
 3447754607,
 3450747437,
 3450712891,
 3450547746,
 3450466450,
 3450492987,
 3450441263,
 3450404917,
 3450400229,
 3450313949,
 3450257864,
 3450723915,
 3450507919,
 3450438433,
 3450453305,
 3450430331,
 3450415751,
 3450391398,
 3450308916,
 3450344613,
 3450322345,
 3450743021,
 3450547466,
 3450551353,
 3450494718,
 3450392071,
 3450356010,
 3450292637,
 3449634285,
 3449587570,
 3449531448,
 3450685998,
 3450504277,
 3450488827,
 3450434228,
 3450397837,
 3450363444,
 3450298965,
 3450314853,
 3450341578,
 3450267034,
 3450719120,
 3450743521,
 3450267904,
 3450217738,
 3450173721,
 3449990242,
 3449934453,
 3449824903,
 3449779717,
 3449792909,
 3450717640,
 3450761926,
 3450679777,
 3450524882,
 3450561670,
 3450489077,
 3450495997,
 3450434077,
 3450450209,
 3450396448,
 3450764574,
 3450546320,
 3450489644,
 3450485093,
 3450419518,
 3450385487,
 3450423544,
 3450307201,
 3450256252,
 3450221287,
 3450081089,
 3450064973,
 3450060654,
 3449588432,
 3449524933,
 3449570421,
 3449474828,
 3449390491,
 3449344772,
 3449320889,
 3450745213,
 3450501487,
 3450499950,
 3450475127,
 3450460987,
 3449390155,
 3448487044,
 3448470275,
 3448332699,
 3448127678,
 3450880668,
 3450540097,
 3450477757,
 3450316262,
 3450235904,
 3449933279,
 3449920933,
 3449708688,
 3449693619,
 3449401356,
 3450536452,
 3449443119,
 3449359211,
 3449352756,
 3449283173,
 3449215726,
 3449062275,
 3447781067,
 3447708703,
 3447704831,
 3450821220,
 3450691658,
 3450518481,
 3450554308,
 3450437673,
 3450433276,
 3450406347,
 3450365284,
 3450309611,
 3450312211,
 3450550457,
 3450457741,
 3450453450,
 3450324294,
 3449953347,
 3449927983,
 3449921065,
 3449866436,
 3449855422,
 3449822600,
 3450726058,
 3450557519,
 3450521179,
 3450006170,
 3449974547,
 3449943979,
 3449941236,
 3449888385,
 3449934554,
 3449921507,
 3450518907,
 3450553997,
 3450479281,
 3450431015,
 3450394734,
 3450339508,
 3450327442,
 3450303576,
 3450181579,
 3450136108,
 3450520634,
 3450447283,
 3450463197,
 3449313313,
 3449359788,
 3449258895,
 3449304018,
 3449271549,
 3447593652,
 3447591603,
 3436823979,
 3436531223,
 3436452484,
 3436374543,
 3436320406,
 3435135483,
 3434983378,
 3434452159,
 3434399905,
 3434043352,
 3450101980,
 3450100266,
 3450029649,
 3450046779,
 3450026328,
 3450053850,
 3450032165,
 3450029834,
 3450006803,
 3449985310,
 3450382934,
 3450318057,
 3450330352,
 3450276826,
 3450282825,
 3450220683,
 3450206308,
 3449953875,
 3449879678,
 3449894101,
 3450860688,
 3450714609,
 3449195823,
 3448904384,
 3448910568,
 3448873886,
 3448779327,
 3447448676,
 3447454704,
 3447389375,
 3450736795,
 3450559773,
 3450466207,
 3450434622,
 3450397049,
 3450074924,
 3450072553,
 3450012034,
 3449978018,
 3449956535]

In [7]:
key = 'RGAPI-db3adf2f-3924-4024-ab6b-c4122163ad71'
watcher = LolWatcher(key)
my_region = 'na1'

In [8]:
# Scraping individual champion score
r1 = requests.get("https://www.metasrc.com/5v5/na/tierlist")
soup1 = BeautifulSoup(r1.text, "html.parser")
power_level = soup1.findAll("div", {"class": "_9581uw"})
names = soup1.findAll("div", {"class": "_q8ue62"})
champ = []
for name in names:
    champ.append(name.getText())
rates = []
for rate in power_level:
    rates.append(rate.getText())
tierlist = pd.DataFrame().assign(
    champ = champ,
    score = rates
)
tierlist.sort_values(by = ["champ"])

# Getting individual Champion IDs
IDs = pd.read_csv("new.txt", sep = ":", header = None)
IDs.columns = ["ID", "champ"]

champion_list = tierlist.sort_values(by = ["champ"])["champ"].unique()

# Merging Ids and individual champion score
new_tierlist = []

# Averaging similar champions scores into one
for champ in champion_list:
    row = {}
    champ_list = tierlist[tierlist["champ"] == champ]
    champ_list_avg = champ_list["score"].astype(float).mean()
    row["champ"] = champ
    row["score"] = champ_list_avg
    new_tierlist.append(row)

tierlist_pd = pd.DataFrame(new_tierlist)
tierlist_pd = tierlist_pd.sort_values(by = ["champ"])
champion_score = IDs.merge(tierlist_pd, on = champion_list)
champion_score = champion_score.drop(columns = ["champ_x", "champ_y"])
champion_score.columns = ["champion", "champion_id", "score"]
champion_score = champion_score.set_index(["champion_id"])

champion_score

,champion,score
champion_id,,
266,Aatrox,48.350
103,Ahri,62.480
84,Akali,37.405
12,Alistar,51.350
32,Amumu,51.140
...,...,...
238,Zed,67.140
115,Ziggs,40.090
26,Zilean,47.155


In [10]:
input_matches = []
i = 0
j = 0

# for ID in gameIDs[:40]:
#     print(i)
#     i += 1
#     match_info = watcher.match.by_id(my_region, ID)
#     input_matches.append(match_info)
    
for i in range(len(gameIDs) % 40):
    for ID in gameIDs[i:i+40]:
        print(j)
        j += 1
        match_info = watcher.match.by_id(my_region, ID)
        input_matches.append(match_info)
        time.sleep(1)
    time.sleep(120)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79


HTTPError: 404 Client Error: Not Found for url: https://na1.api.riotgames.com/lol/match/v4/matches/4645085775